In [1]:
import pandas as pd
import tensorflow as tf
import sklearn 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df=r"C:\Users\nupur\python data science\datasets\Churn_Modelling.csv"
df=pd.read_csv(df)
X=df.iloc[:,4:13]
Y=df.iloc[:,-1]
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X["Gender"],drop_first=True)
X=X.drop(["Geography","Gender"],axis=1)
X=pd.concat([X,geography,gender],axis=1)
x_train,x_test,y_train,y_test=train_test_split(X,Y,random_state=42,test_size=0.2)
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout

In [4]:
def build_model(hp):
    model=keras.Sequential()
    for i in range(hp.Int("num_layers",2,30)):
        model.add(layers.Dense(units=hp.Int("unit_"+str(i),min_value=32,max_value=512,step=32),activation="relu"))
    model.add(layers.Dense(1,activation="sigmoid"))   
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])),loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [6]:
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='project1',
    project_name='Churn')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Reloading Tuner from project1\Churn\tuner0.json


In [7]:
tuner.search(x_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

In [8]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)

In [9]:
history=model.fit(x_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7929 - loss: 0.4841 - val_accuracy: 0.8575 - val_loss: 0.3490
Epoch 2/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8496 - loss: 0.3606 - val_accuracy: 0.8587 - val_loss: 0.3443
Epoch 3/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8607 - loss: 0.3464 - val_accuracy: 0.8569 - val_loss: 0.3396
Epoch 4/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8661 - loss: 0.3348 - val_accuracy: 0.8587 - val_loss: 0.3453
Epoch 5/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8674 - loss: 0.3292 - val_accuracy: 0.8531 - val_loss: 0.3557
Epoch 6/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8742 - loss: 0.3209 - val_accuracy: 0.8525 - val_loss: 0.3550
Epoch 7/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8758 - loss: 0.3097 - val_accuracy: 0.8494 - val_loss: 0.3534
Epoch 8/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8685 - loss: 0.3209 - val_accuracy: 0.

In [10]:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Best epoch: 2


In [11]:
model = tuner.hypermodel.build(best_hps)

# Retrain the model
s=model.fit(x_train, y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/2
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7995 - loss: 0.4771 - val_accuracy: 0.8575 - val_loss: 0.3596
Epoch 2/2
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8614 - loss: 0.3604 - val_accuracy: 0.8575 - val_loss: 0.3541


In [12]:
eval_result = model.evaluate(x_test, y_test)
print("[test loss, test accuracy]:", eval_result)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8600 - loss: 0.3517
[test loss, test accuracy]: [0.3498908579349518, 0.8629999756813049]
